In [1]:
import pandas as pd
import os
import numpy as np
import requests
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime, timedelta
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data, process_weather_data
from get_electricity_prices import get_data
from get_weather_data import get_historical_weather, get_weather_forecast
from entsoe_data import fetch_historical_data, ensure_valid_series



In [2]:
load_dotenv()
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")
entose_api = os.getenv("ENTSOE_API")

os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
fs = project.get_feature_store() 
print(f"Connected to project: {project.name}")

2025-01-05 17:06:03,482 INFO: Initializing external client
2025-01-05 17:06:03,483 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 17:06:04,991 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164446
Connected to project: oskaralf


In [3]:
start_date = datetime.now().strftime('%Y-%m-%d')
end_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
forecast = get_weather_forecast("Stockholm", "2022-11-01", "2025-01-03", 59.3294, 18.0687)

formatted_forecast_df = process_weather_data(forecast)
print(formatted_forecast_df.dtypes)

Coordinates 59.32889938354492°N 18.072357177734375°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
date                   datetime64[ns]
temperature_2m                float32
precipitation                 float32
snow_depth                    float32
pressure_msl                  float32
cloud_cover                   float32
wind_speed_10m                float32
wind_speed_100m               float32
wind_direction_10m            float32
wind_direction_100m           float32
city                           object
dtype: object


In [7]:
# price_fg = fs.get_feature_group(
#     name='electricity_price_data_3',
#     version=1,
# )
weather_fg = fs.get_feature_group(
    name='weather_data_3',
    version=1,
)
entsoe_fg = fs.get_feature_group(
    name='entsoe_data_3',
    version=1,
)

In [8]:
weather_fg.insert(formatted_forecast_df)

2025-01-05 17:06:58,863 INFO: 	5 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394593


Uploading Dataframe: 100.00% |██████████| Rows 168/168 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_3_1_offline_fg_materialization/executions


(Job('weather_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation",
           "min_value": -5.0,
           "max_value": 500.0
         },
         "meta": {
           "expectationId": 694432
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 168,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-05T04:06:58.000862Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
       

In [9]:
# price = get_todays_data("SE3")
# price['time_start'] = price['time_start'].str.replace(r'\+\d{2}:\d{2}$', '', regex=True)
# print(price)
# print(price.dtypes)

In [10]:
entsoe_df = fetch_historical_data(entose_api, start_date, end_date)
entsoe_df.columns = entsoe_df.columns.str.lower()
entsoe_df.columns = entsoe_df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
entsoe_df['date'] = pd.to_datetime(entsoe_df['date'])
print(entsoe_df.dtypes)
print(entsoe_df.tail(5))

[DEBUG] Successfully fetched load data for finland.
[DEBUG] Successfully fetched load data for norway.
[DEBUG] Successfully fetched load data for denmark.
[DEBUG] Successfully fetched cross-border flows SE3 to finland.
[DEBUG] Successfully fetched cross-border flows finland to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to norway.
[DEBUG] Successfully fetched cross-border flows norway to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to denmark.
[DEBUG] Successfully fetched cross-border flows denmark to SE3.
load_se3                                                   float64
load_finland                                               float64
load_norway                                                float64
load_denmark                                               float64
prices                                                     float64
total_generation_biomass                                   float64
total_generation_fossil_gas                                fl

In [11]:
#print(entsoe_df['date'].tail(20))

In [16]:
# print(entose_api)
# entsoe_df = fetch_energy_data(entose_api, "2025-01-05", "2025-01-06")

# print(entsoe_df)

In [12]:
entsoe_fg.insert(entsoe_df)

2025-01-05 17:10:02,821 INFO: 	21 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394594


Uploading Dataframe: 100.00% |██████████| Rows 16/16 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: entsoe_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/entsoe_data_3_1_offline_fg_materialization/executions


(Job('entsoe_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "load_finland",
           "min_value": -0.1,
           "max_value": 20000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 694442
         }
       },
       "result": {
         "observed_value": 10617.25,
         "element_count": 16,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-05T04:10:02.000820Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_c

In [11]:
# price_fg.insert(price)